In [ ]:
import boto3
from strands.models import BedrockModel
from botocore.config import Config as BotocoreConfig
from strands import Agent
from pydantic import BaseModel
import dotenv
import os
from strands import Agent, tool
import base64
import json
dotenv.load_dotenv(".env", override=True)
# Create a custom boto3 session
session = boto3.Session(
    aws_access_key_id= os.environ.get('AWS_ACCESS_KEY'),
    aws_secret_access_key= os.environ.get('AWS_SECRET_KEY'),
    region_name= 'ap-southeast-1'
    
)
boto_config = BotocoreConfig(
    retries={"max_attempts": 3, "mode": "standard"},
    connect_timeout=5,
    read_timeout=60
)
# Create a Bedrock model with the custom session
bedrock_model = BedrockModel(
    model_id="arn:aws:bedrock:ap-southeast-1:389903776084:inference-profile/apac.anthropic.claude-3-5-sonnet-20240620-v1:0",
    boto_session=session,
    temperature=0.7,
    top_p=0.8,
    stop_sequences=["###", "END"],
    boto_client_config=boto_config,
)

client = session.client("bedrock-runtime")

with open("test.pdf", "rb") as f:
    img_b64 = base64.b64encode(f.read()).decode("utf-8")
    
payload = {
    "system": [{"type": "text", "text": "Bạn là chuyên gia tài chính, hãy trích thông tin chính xác từ ảnh báo cáo tài chính."}] ,
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Please summarize the content of this document page."},
                {"type": "pdf", "source": {"type": "base64", "media_type": "application/pdf", "data": img_b64}}
            ]
        }
    ],
    # "max_tokens": 1024,
    # "anthropic_version": "bedrock-2023-05-31"
}

response = client.invoke_model(
    modelId="arn:aws:bedrock:ap-southeast-1:389903776084:inference-profile/apac.amazon.nova-pro-v1:0",
    contentType="application/json",
    accept="application/json",
    body=json.dumps(payload)
)

print(response["body"].read().decode())

defaulted to us-west-2 because no region was specified
issue=<https://github.com/strands-agents/sdk-python/issues/238> | this behavior will change in an upcoming release


ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: Malformed input request: #/system: expected type: JSONArray, found: String, please reformat your input and try again.

In [25]:
from chartool import analyze_financial_data
# from strands_tools import


class TaiChinh(BaseModel):
    status: bool
    error: str
    content: str
    thongso: str
    diemmanh: str
    diemyeu: str
    ruiro: str
    ketluan: str
    
agent = Agent(model = bedrock_model, tools=[analyze_financial_data], system_prompt="""
You are a financial analysis expert. 

You can use the analyze_financial_data tool to calculate key financial metrics such as ROE, ROA, ROS, Debt Ratio,...  automatically generate insightful analytical charts.

You are provided with the company’s full quarterly financial data in the form of a JSON list. For example:

[
  {
    "quy": "Q1/2025",
    "tong_tai_san": 1000000,
    "tong_no": 800000,
    "von_chu_so_huu": 200000,
    "tong_doanh_thu": 500000,
    "loi_nhuan_sau_thue": 25000,
    "loi_nhuan_truoc_thue": 30000,
    "loi_nhuan_gop": 80000,
    "tong_tai_san_luu_dong_ngan_han": 400000,
    "no_ngan_han": 250000
  },
  ...
]

Your task includes the following:

    Always thinking clearly and including <think></think> to show step in your response

    Calculate and report key financial indicators (ROE, ROA, ROS, Debt Ratio, CurrentRatio, GrossProfitMargin, AssetTurnoverRatio, etc.) for each quarter.

    Generate charts to visualize trends and changes over time.

    Provide a clear and structured financial analysis, highlighting:

        Significant changes between quarters.

        Strengths and weaknesses in the company’s financial position.

        Potential risks based on financial ratios and trends.


Your report should be accurate, insightful, and visually supported by charts for better decision-making.
Your report must contains reasoning step that you have done
Report in Vietnamese with well structured and clearly. 

""")

output = agent("""{
        "quy": "Q1/2025",
        "tong_tai_san_cuoi_quy": 236156479630,
        "loi_nhuan_sau_thue": 1116027647,
        "loi_nhuan_gop": 2304572679,
        "von_chu_so_huu": 178445053214,
        "tong_doanh_thu": 60107916752,
        "tong_tai_san": 236156479630,
        "tong_no": 57711426416,
        "gia_von_hang_ban": 57803344073,
        "loi_nhuan_gop_ve_BH_va_CCDV": 2304572679,
        "loi_nhuan_tai_chinh": 622690374,
        "loi_nhuan_truoc_thue": 1402037633,
        "tong_tai_san_luu_dong_ngan_han": 172374527886,
        "no_ngan_han": 57711426416
      },
      {
        "quy": "Q2/2024",
        "tong_tai_san_cuoi_quy": 212146243168,
        "loi_nhuan_sau_thue": 10588556972,
        "loi_nhuan_gop": 2283265819,
        "von_chu_so_huu": 167262603675,
        "tong_doanh_thu": 42083671325,
        "tong_tai_san": 212146243168,
        "tong_no": 44883639493,
        "gia_von_hang_ban": 39800405506,
        "loi_nhuan_gop_ve_BH_va_CCDV": 2283265819,
        "loi_nhuan_tai_chinh": 189638689,
        "loi_nhuan_truoc_thue": 13259726458,
        "tong_tai_san_luu_dong_ngan_han": 145623535885,
        "no_ngan_han": 44883639493
      },
      {
        "quy": "Q3/2024",
        "tong_tai_san_cuoi_quy": 227375689868,
        "loi_nhuan_sau_thue": 13875548792,
        "loi_nhuan_gop": 1791245409,
        "von_chu_so_huu": 181138152467,
        "tong_doanh_thu": 41977528911,
        "tong_tai_san": 227375689868,
        "tong_no": 46237537401,
        "gia_von_hang_ban": 40186283502,
        "loi_nhuan_gop_ve_BH_va_CCDV": 1791245409,
        "loi_nhuan_tai_chinh": 834803115,
        "loi_nhuan_truoc_thue": 17204922944,
        "tong_tai_san_luu_dong_ngan_han": 162255806260,
        "no_ngan_han": 46237537401
      },
      {
        "quy": "Q4/2024",
        "tong_tai_san_cuoi_quy": 229773312284,
        "loi_nhuan_sau_thue": -3809126900,
        "loi_nhuan_gop": 2114408986,
        "von_chu_so_huu": 177329025567,
        "tong_doanh_thu": 62287700880,
        "tong_tai_san": 229773312284,
        "tong_no": 52444286717,
        "gia_von_hang_ban": 60173291894,
        "loi_nhuan_gop_ve_BH_va_CCDV": 2114408986,
        "loi_nhuan_tai_chinh": 617879595,
        "loi_nhuan_truoc_thue": -513742519,
        "tong_tai_san_luu_dong_ngan_han": 164800586261,
        "no_ngan_han": 52444286717
      }""")

Để phân tích tình hình tài chính của công ty dựa trên dữ liệu được cung cấp, tôi sẽ sử dụng công cụ analyze_financial_data để tính toán các chỉ số tài chính quan trọng và tạo biểu đồ phân tích. Sau đó, tôi sẽ cung cấp một báo cáo chi tiết về tình hình tài chính của công ty.

<think>
1. Chuẩn bị dữ liệu đầu vào cho công cụ analyze_financial_data
2. Gọi công cụ để tính toán các chỉ số và tạo biểu đồ
3. Phân tích kết quả và viết báo cáo
</think>

Đầu tiên, tôi sẽ gọi công cụ analyze_financial_data:
Tool #1: analyze_financial_data
Saved chart: charts\tai_san_va_hieu_qua_sinh_loi.png
Saved chart: charts\von_chu_va_sinh_loi.png
Saved chart: charts\doanh_thu_va_hieu_qua.png
Saved chart: charts\bien_loi_nhuan_va_loi_nhuan.png
Saved chart: charts\co_cau_tai_san_no.png
Saved chart: charts\phan_tich_loi_nhuan.png
Saved chart: charts\co_cau_von.png
Saved chart: charts\hieu_suat_va_kha_nang_thanh_toan.png
{'ResponseMetadata': {'RequestId': '77534a13-6e96-4a06-a71f-c2ef189c2d5d', 'HTTPStatusCode': 2

In [26]:
str(output)

'\n\nDựa trên kết quả phân tích từ công cụ analyze_financial_data, tôi sẽ cung cấp một báo cáo chi tiết về tình hình tài chính của công ty:\n\n<think>\n1. Phân tích các chỉ số tài chính chính\n2. Xem xét biến động giữa các quý\n3. Đánh giá điểm mạnh và điểm yếu của công ty\n4. Xác định các rủi ro tiềm ẩn\n5. Tổng hợp và đưa ra kết luận\n</think>\n\nBáo cáo Phân tích Tài chính\n\n1. Tổng quan:\n\nDữ liệu tài chính được cung cấp cho 4 quý, từ Q2/2024 đến Q1/2025. Qua phân tích, chúng ta có thể thấy công ty đã trải qua những biến động đáng kể trong hiệu quả hoạt động và tình hình tài chính.\n\n2. Phân tích các chỉ số tài chính chính:\n\na) Khả năng sinh lời:\n- ROE (Tỷ suất lợi nhuận trên vốn chủ sở hữu):\n  Q2/2024: 6.33%\n  Q3/2024: 7.66%\n  Q4/2024: -2.15%\n  Q1/2025: 0.63%\n\n- ROA (Tỷ suất lợi nhuận trên tổng tài sản):\n  Q2/2024: 4.99%\n  Q3/2024: 6.10%\n  Q4/2024: -1.66%\n  Q1/2025: 0.47%\n\n- ROS (Tỷ suất lợi nhuận trên doanh thu):\n  Q2/2024: 25.16%\n  Q3/2024: 33.05%\n  Q4/2024:

In [11]:
output.metrics.tool_metrics.get('analyze_financial_data','')

ToolMetrics(tool={'toolUseId': 'tooluse_VfwjnClkT6GxlVQZtQEQ0Q', 'name': 'analyze_financial_data', 'input': {'quarterly_data': [{'quy': 'Q1/2025', 'tong_tai_san_cuoi_quy': 236156479630, 'loi_nhuan_sau_thue': 1116027647, 'loi_nhuan_gop': 2304572679, 'von_chu_so_huu': 178445053214, 'tong_doanh_thu': 60107916752, 'tong_tai_san': 236156479630, 'tong_no': 57711426416, 'gia_von_hang_ban': 57803344073, 'loi_nhuan_gop_ve_BH_va_CCDV': 2304572679, 'loi_nhuan_tai_chinh': 622690374, 'loi_nhuan_truoc_thue': 1402037633, 'tong_tai_san_luu_dong_ngan_han': 172374527886, 'no_ngan_han': 57711426416}, {'quy': 'Q4/2024', 'tong_tai_san_cuoi_quy': 229773312284, 'loi_nhuan_sau_thue': -3809126900, 'loi_nhuan_gop': 2114408986, 'von_chu_so_huu': 177329025567, 'tong_doanh_thu': 62287700880, 'tong_tai_san': 229773312284, 'tong_no': 52444286717, 'gia_von_hang_ban': 60173291894, 'loi_nhuan_gop_ve_BH_va_CCDV': 2114408986, 'loi_nhuan_tai_chinh': 617879595, 'loi_nhuan_truoc_thue': -513742519, 'tong_tai_san_luu_dong_nga

In [7]:
tool_logs = []

# Nếu bạn muốn lấy log đã lưu từ công cụ
if output.metrics.tool_metrics.get("analyze_financial_data"):
    tool_result = output.metrics.tool_metrics["analyze_financial_data"].tool
    tool_input = tool_result.get("input", {})
    saved_paths = tool_result.get("output", {}).get("saved_chart_paths", [])  # nếu bạn custom trả về paths

    tool_logs = [f"Saved chart: {p}" for p in saved_paths]

In [8]:
tool_logs

[]

In [4]:
from doc_extr import doc_extr_s3
import json 
data = doc_extr_s3(bucket="testworkflow123", prefix="pdf/vlg/")
print(json.dumps(data, indent=2, ensure_ascii=False))

[
  {
    "company": {
      "TenCongTy": "CÔNG TY CỔ PHẦN VIMC LOGISTICS",
      "MaChungKhoan": "VLG",
      "DiaChi": "Tầng 4, tòa nhà Ocean Park, số 1 Đào Duy Anh, phường Phương Mai, quận Đống Đa, TP.Hà Nội, Việt Nam",
      "SoDienThoai": "04-35772047/48",
      "Email": "info@vimclogistics.vn",
      "LinhVucKinhDoanh": "Kinh doanh xuất nhập khẩu hàng hóa, môi giới thương mại, đại lý mua bán, ký gửi hàng hóa; Vận tải đa phương thức; vận tải hàng hóa, container, hàng hóa siêu trường, siêu trọng bằng đường bộ, đường sắt, đường biển"
    },
    "bctc": [
      {
        "quy": "Q1/2025",
        "tong_tai_san_cuoi_quy": 236156479630,
        "loi_nhuan_sau_thue": 1116027647,
        "loi_nhuan_gop": 2304572679,
        "von_chu_so_huu": 178445053214,
        "tong_doanh_thu": 60107916752,
        "tong_tai_san": 236156479630,
        "tong_no": 57711426416,
        "gia_von_hang_ban": 57803344073,
        "loi_nhuan_gop_ve_BH_va_CCDV": 2304572679,
        "loi_nhuan_tai_chinh": 6226

In [5]:
data

({'company': {'TenCongTy': 'CÔNG TY CỔ PHẦN VIMC LOGISTICS',
   'MaChungKhoan': 'VLG',
   'DiaChi': 'Tầng 4, tòa nhà Ocean Park, số 1 Đào Duy Anh, phường Phương Mai, quận Đống Đa, TP.Hà Nội, Việt Nam',
   'SoDienThoai': '04-35772047/48',
   'Email': 'info@vimclogistics.vn',
   'LinhVucKinhDoanh': 'Kinh doanh xuất nhập khẩu hàng hóa, môi giới thương mại, đại lý mua bán, ký gửi hàng hóa; Vận tải đa phương thức; vận tải hàng hóa, container, hàng hóa siêu trường, siêu trọng bằng đường bộ, đường sắt, đường biển'},
  'bctc': [{'quy': 'Q1/2025',
    'tong_tai_san_cuoi_quy': 236156479630,
    'loi_nhuan_sau_thue': 1116027647,
    'loi_nhuan_gop': 2304572679,
    'von_chu_so_huu': 178445053214,
    'tong_doanh_thu': 60107916752,
    'tong_tai_san': 236156479630,
    'tong_no': 57711426416,
    'gia_von_hang_ban': 57803344073,
    'loi_nhuan_gop_ve_BH_va_CCDV': 2304572679,
    'loi_nhuan_tai_chinh': 622690374,
    'loi_nhuan_truoc_thue': 1402037633,
    'tong_tai_san_luu_dong_ngan_han': 17237452

In [7]:
structure_agent = Agent(model =bedrock_model)

In [12]:
from dataclasses import dataclass, asdict
from typing import List
@dataclass
class CompanyInfo:
    ten_cty: str
    ma_ck: str
    dia_chi: str
    so_dt: str
    email: str
    linhvuckinhdoanh: str

@dataclass
class BctcItem:
    quy: str
    tong_tai_san_cuoi_quy: int
    loi_nhuan_sau_thue: int
    loi_nhuan_gop: int
    tong_doanh_thu: int
    tong_tai_san: int
    tong_no: int
    gia_von_hang_ban: int
    loi_nhuan_gop_ve_BH_va_CCDV: int
    loi_nhuan_tai_chinh: int
    loi_nhuan_truoc_thue: int
    tong_tai_san_luu_dong_ngan_han: int
    no_ngan_han: int


class CongTy(BaseModel):
    status: bool
    error: str
    companyInfo: CompanyInfo
    bctc: List[BctcItem]
out = json.dumps(data, ensure_ascii=False)
cty_extract = structure_agent.structured_output(CongTy, f"""
Hãy phân tích và sắp xếp dữ liệu lại đặc biệt về dữ liệu báo cáo tài chính, sắp xếp lại theo list các quý ứng với các biến sắp xếp theo chiều tăng dần về thời gian.
Trả về status: False khi không trích xuất được đủ trường cho 4 quý, và error khi status là False sẽ chỉ ra các trường còn thiếu. 
{out}
""")

Tôi sẽ phân tích và sắp xếp dữ liệu báo cáo tài chính của công ty VIMC LOGISTICS theo thứ tự thời gian tăng dần.
Tool #3: CongTy


In [14]:
cty_extract.companyInfo

CompanyInfo(ten_cty='CÔNG TY CỔ PHẦN VIMC LOGISTICS', ma_ck='VLG', dia_chi='Tầng 4, tòa nhà Ocean Park, số 1 Đào Duy Anh, phường Phương Mai, quận Đống Đa, TP.Hà Nội, Việt Nam', so_dt='04-35772047/48', email='info@vimclogistics.vn', linhvuckinhdoanh='Kinh doanh xuất nhập khẩu hàng hóa, môi giới thương mại, đại lý mua bán, ký gửi hàng hóa; Vận tải đa phương thức; vận tải hàng hóa, container, hàng hóa siêu trường, siêu trọng bằng đường bộ, đường sắt, đường biển')

In [15]:
json.dumps(asdict(cty_extract.companyInfo), ensure_ascii=False)

'{"ten_cty": "CÔNG TY CỔ PHẦN VIMC LOGISTICS", "ma_ck": "VLG", "dia_chi": "Tầng 4, tòa nhà Ocean Park, số 1 Đào Duy Anh, phường Phương Mai, quận Đống Đa, TP.Hà Nội, Việt Nam", "so_dt": "04-35772047/48", "email": "info@vimclogistics.vn", "linhvuckinhdoanh": "Kinh doanh xuất nhập khẩu hàng hóa, môi giới thương mại, đại lý mua bán, ký gửi hàng hóa; Vận tải đa phương thức; vận tải hàng hóa, container, hàng hóa siêu trường, siêu trọng bằng đường bộ, đường sắt, đường biển"}'

In [4]:
output = {'company': {'TenCongTy': 'CÔNG TY CỔ PHẦN VIMC LOGISTICS',
  'MaChungKhoan': 'VLG',
  'DiaChi': 'Phòng 806, tòa nhà Ocean Park, số 1 Đào Duy Anh, phường Phương Mai, quận Đống Đa, TP.Hà Nội, Việt Nam',
  'SoDienThoai': '024-35772047/48',
  'Email': 'info@vimclogistics.vn',
  'LinhVucKinhDoanh': 'Kinh doanh xuất nhập khẩu hàng hóa, môi giới thương mại, đại lý mua bán, ký gửi hàng hóa; Vận tải đa phương thức; vận tải hàng hóa, container, hàng hóa siêu trường, siêu trọng bằng đường bộ, đường sắt, đường biển'},
 'bctc': [{'quy': 'Q1/2025',
   'tong_tai_san_cuoi_quy': 236156479630,
   'loi_nhuan_sau_thue': 1116027647,
   'loi_nhuan_gop': 2304572679,
   'von_chu_so_huu': 178445053214,
   'tong_doanh_thu': 60107916752,
   'tong_tai_san': 236156479630,
   'tong_no': 57711426416,
   'gia_von_hang_ban': 57803344073,
   'loi_nhuan_gop_ve_BH_va_CCDV': 2304572679,
   'loi_nhuan_tai_chinh': 622690374,
   'loi_nhuan_truoc_thue': 1402037633,
   'tong_tai_san_luu_dong_ngan_han': 172374527886,
   'no_ngan_han': 57711426416},
  {'quy': 'Q2/2024',
   'tong_tai_san_cuoi_quy': 212146243168,
   'loi_nhuan_sau_thue': 10588556972,
   'loi_nhuan_gop': 2283265819,
   'von_chu_so_huu': 167262603675,
   'tong_doanh_thu': 42083671325,
   'tong_tai_san': 212146243168,
   'tong_no': 44883639493,
   'gia_von_hang_ban': 39800405506,
   'loi_nhuan_gop_ve_BH_va_CCDV': 2283265819,
   'loi_nhuan_tai_chinh': 190238689,
   'loi_nhuan_truoc_thue': 13259726458,
   'tong_tai_san_luu_dong_ngan_han': 145623535885,
   'no_ngan_han': 44883639493},
  {'quy': 'Q3/2024',
   'tong_tai_san_cuoi_quy': 227375689868,
   'loi_nhuan_sau_thue': 13875548792,
   'loi_nhuan_gop': 1791245409,
   'von_chu_so_huu': 181138152467,
   'tong_doanh_thu': 41977528911,
   'tong_tai_san': 227375689868,
   'tong_no': 46237537401,
   'gia_von_hang_ban': 40186283502,
   'loi_nhuan_gop_ve_BH_va_CCDV': 1791245409,
   'loi_nhuan_tai_chinh': 834803115,
   'loi_nhuan_truoc_thue': 17204922944,
   'tong_tai_san_luu_dong_ngan_han': 162255806260,
   'no_ngan_han': 46237537401},
  {'quy': 'Q4/2024',
   'tong_tai_san_cuoi_quy': 229773312284,
   'loi_nhuan_sau_thue': -3809126900,
   'loi_nhuan_gop': 2114408986,
   'von_chu_so_huu': 177329025567,
   'tong_doanh_thu': 62287700880,
   'tong_tai_san': 229773312284,
   'tong_no': 52444286717,
   'gia_von_hang_ban': 60173291894,
   'loi_nhuan_gop_ve_BH_va_CCDV': 2114408986,
   'loi_nhuan_tai_chinh': 617879595,
   'loi_nhuan_truoc_thue': -513742519,
   'tong_tai_san_luu_dong_ngan_han': 164800586261,
   'no_ngan_han': 52444286717}]}

In [ ]:
agent = Agent(model = bedrock_model, )
agent("Hello")

Hello! How are you doing today? Is there anything I can help you with?

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'Hello! How are you doing today? Is there anything I can help you with?'}]}, metrics=EventLoopMetrics(cycle_count=1, tool_metrics={}, cycle_durations=[1.7326014041900635], traces=[<strands.telemetry.metrics.Trace object at 0x000001E8D62F6080>], accumulated_usage={'inputTokens': 8, 'outputTokens': 20, 'totalTokens': 28}, accumulated_metrics={'latencyMs': 1575}), state={})

In [3]:
from strands import Agent

structure_agent = Agent(model =bedrock_model)
trichxuat = structure_agent.structured_output(CongTy, f"""
Hãy phân tích và sắp xếp dữ liệu lại đặc biệt về dữ liệu báo cáo tài chính, sắp xếp lại theo list các quý ứng với các biến sắp xếp theo chiều tăng dần về thời gian.
Trả về status: False khi không trích xuất được đủ trường cho 4 quý
{json.dumps(output, ensure_ascii=False)}
                                           """)

NameError: name 'CongTy' is not defined

In [6]:
trichxuat.bctc

[BctcItem(quy='Q2/2024', tong_tai_san_cuoi_quy=212146243168, loi_nhuan_sau_thue=10588556972, loi_nhuan_gop=2283265819, tong_doanh_thu=42083671325, tong_tai_san=212146243168, tong_no=44883639493, gia_von_hang_ban=39800405506, loi_nhuan_gop_ve_BH_va_CCDV=2283265819, loi_nhuan_tai_chinh=190238689, loi_nhuan_truoc_thue=13259726458, tong_tai_san_luu_dong_ngan_han=145623535885, no_ngan_han=44883639493),
 BctcItem(quy='Q3/2024', tong_tai_san_cuoi_quy=227375689868, loi_nhuan_sau_thue=13875548792, loi_nhuan_gop=1791245409, tong_doanh_thu=41977528911, tong_tai_san=227375689868, tong_no=46237537401, gia_von_hang_ban=40186283502, loi_nhuan_gop_ve_BH_va_CCDV=1791245409, loi_nhuan_tai_chinh=834803115, loi_nhuan_truoc_thue=17204922944, tong_tai_san_luu_dong_ngan_han=162255806260, no_ngan_han=46237537401),
 BctcItem(quy='Q4/2024', tong_tai_san_cuoi_quy=229773312284, loi_nhuan_sau_thue=-3809126900, loi_nhuan_gop=2114408986, tong_doanh_thu=62287700880, tong_tai_san=229773312284, tong_no=52444286717, gia

In [7]:
from dataclasses import dataclass, asdict
json_data = [json.dumps(asdict(item), ensure_ascii=False) for item in trichxuat.bctc]

In [8]:
json_data

['{"quy": "Q2/2024", "tong_tai_san_cuoi_quy": 212146243168, "loi_nhuan_sau_thue": 10588556972, "loi_nhuan_gop": 2283265819, "tong_doanh_thu": 42083671325, "tong_tai_san": 212146243168, "tong_no": 44883639493, "gia_von_hang_ban": 39800405506, "loi_nhuan_gop_ve_BH_va_CCDV": 2283265819, "loi_nhuan_tai_chinh": 190238689, "loi_nhuan_truoc_thue": 13259726458, "tong_tai_san_luu_dong_ngan_han": 145623535885, "no_ngan_han": 44883639493}',
 '{"quy": "Q3/2024", "tong_tai_san_cuoi_quy": 227375689868, "loi_nhuan_sau_thue": 13875548792, "loi_nhuan_gop": 1791245409, "tong_doanh_thu": 41977528911, "tong_tai_san": 227375689868, "tong_no": 46237537401, "gia_von_hang_ban": 40186283502, "loi_nhuan_gop_ve_BH_va_CCDV": 1791245409, "loi_nhuan_tai_chinh": 834803115, "loi_nhuan_truoc_thue": 17204922944, "tong_tai_san_luu_dong_ngan_han": 162255806260, "no_ngan_han": 46237537401}',
 '{"quy": "Q4/2024", "tong_tai_san_cuoi_quy": 229773312284, "loi_nhuan_sau_thue": -3809126900, "loi_nhuan_gop": 2114408986, "tong_do

In [9]:
type(json_data)

list

In [10]:
from finance_agent import FinancialAgent

In [11]:
fin_agent = FinancialAgent()

defaulted to us-west-2 because no region was specified
issue=<https://github.com/strands-agents/sdk-python/issues/238> | this behavior will change in an upcoming release


In [ ]:
fin_data = fin_agent.compute(str(json_data))

In [15]:
type(fin_data)

str

In [18]:
class TaiChinh(BaseModel):
    status: bool
    thongso: str
    diemmanh: str
    diemyeu: str
    ruiro: str
    ketluan: str

fin_struct_agent = Agent(model = bedrock_model, system_prompt="You are an extracted agent that extract agent based on financial analyst. Status is true if result show that previous agent actually use tool")
fin_analyst = fin_struct_agent.structured_output(TaiChinh, fin_data)




Tool #1: TaiChinh


In [21]:
fin_analyst.diemmanh

'Khả năng thanh toán ngắn hạn xuất sắc với Current Ratio luôn trên 2.9. Cấu trúc vốn an toàn với tỷ lệ nợ thấp (dưới 25%). Doanh thu tăng trưởng mạnh 48% trong Q4/2024. Tổng tài sản tăng ổn định từ 212 tỷ lên 236 tỷ đồng. Vốn chủ sở hữu duy trì ổn định khoảng 178 tỷ đồng.'

In [22]:
fin_sum = f"""
Điểm mạnh: {fin_analyst.diemmanh}
Điểm yếu: {fin_analyst.diemyeu}
Rủi ro: {fin_analyst.ruiro}
"""

In [1]:
import boto3
from strands.models import BedrockModel
from botocore.config import Config as BotocoreConfig
from strands import Agent
from pydantic import BaseModel
import dotenv
import os
from strands import Agent, tool
import base64
import json
from strands_tools import retrieve
dotenv.load_dotenv(".env", override=True)
# Create a custom boto3 session
session = boto3.Session(
    aws_access_key_id= os.environ.get('AWS_ACCESS_KEY'),
    aws_secret_access_key= os.environ.get('AWS_SECRET_KEY'),
    region_name= 'ap-southeast-1'
    
)
boto_config = BotocoreConfig(
    retries={"max_attempts": 3, "mode": "standard"},
    connect_timeout=5,
    read_timeout=60
)
# Create a Bedrock model with the custom session
bedrock_model = BedrockModel(
    model_id="arn:aws:bedrock:ap-southeast-1:389903776084:inference-profile/apac.anthropic.claude-3-5-sonnet-20241022-v2:0",
    boto_session=session,
    temperature=0.3,
    top_p=0.8,
    # stop_sequences=["###", "END"],
    # boto_client_config=boto_config,
)

client = session.client("bedrock-runtime")

os.environ["KNOWLEDGE_BASE_ID"] = 'LXTS8BCTDL'
os.environ["AWS_REGION"] = 'ap-southeast-1'
os.environ["MIN_SCORE"] = "0.05"

os.environ["AWS_ACCESS_KEY_ID"] =os.environ.get('AWS_ACCESS_KEY')

os.environ["AWS_SECRET_ACCESS_KEY"] = os.environ.get('AWS_SECRET_KEY')

defaulted to us-west-2 because no region was specified
issue=<https://github.com/strands-agents/sdk-python/issues/238> | this behavior will change in an upcoming release


In [2]:

from totrinh import create_to_trinh
from upload import download_and_insert_image_from_s3
simple_agent = Agent(
    model = bedrock_model,
    tools=[retrieve, create_to_trinh],
    system_prompt = """
🧠 Bạn là một agent hỗ trợ tạo báo cáo tín dụng doanh nghiệp. Hỗ trợ người dùng có thể sửa lại nội dung.

Thay vì điền trực tiếp văn bản, bạn cần sử dụng thông tin từ các trường dữ liệu JSON đầu vào, tổng hợp lại và trình bày thành đoạn văn bản phù hợp.

📦 Cấu trúc dữ liệu JSON đầu vào bao gồm các trường sau:

{
  "customer": "Thông tin chung của khách hàng, bao gồm: tên công ty, mã số thuế, số điện thoại, địa chỉ, email, người đại diện pháp luật.",
  "lich_su_phat_trien": "Lịch sử phát triển của công ty, có thể bao gồm thông tin về công ty mẹ, công ty con.",
  "ban_lanh_dao": "Cơ cấu tổ chức, danh sách và vai trò của các thành viên trong ban lãnh đạo, hội đồng quản trị.",
  "danh_gia_thi_truong": "Đánh giá tình hình thị trường mà công ty đang hoạt động.",
  "danh_gia_san_pham": "Đánh giá sản phẩm, khả năng cạnh tranh và điểm nổi bật của công ty trên thị trường.",
  "danh_gia_kiem_toan": "Tình trạng kiểm toán báo cáo tài chính, đơn vị kiểm toán, nội dung nổi bật của báo cáo. Bạn cần tìm trong dữ liệu JSON trường `kiem_toan` để lấy thông tin `status`.",
  "tinh_hinh_tai_chinh": "Phân tích, trình bày rõ ràng cụ thể tất cả tình hình tài chính của công ty.",
  "ruiro": "Trình bày những rủi ro công ty 1 cách có cấu trúc và đưa ra giải pháp nếu có.",
  "ketluan": "Từ toàn bộ thông tin trong kho dữ liệu, phân tích, đưa ra kết luận về công ty và đề xuất có nên cho công ty vay vốn với tình hình đó không"
}

🛠 Các công cụ sẵn có:

    retrieve: Truy xuất dữ liệu từ kho tài liệu.

    create_to_trinh: Dùng hàm như sau:
    create_to_trinh({'customer': '...', ...})
    (Chỉ giữ nguyên nội dung bên trong các trường JSON sau khi tổng hợp thành đoạn văn.)

    create_content: Tạo văn bản từ các nội dung đưa vào.

<guideline>

1. Dùng công cụ `retrieve` để lấy toàn bộ dữ liệu cần thiết từ hệ thống.

2. Với từng trường dữ liệu nhận được:

    - Phân tích và viết lại nội dung thành một đoạn văn ngắn, khoảng 200–300 từ.
    - Trình bày văn bản rõ ràng, chuyên nghiệp theo phong cách báo cáo tín dụng.
    - **Bắt buộc**: Sau mỗi ý hoặc đoạn chính, cần **xuống dòng bằng `\\n\\n`** để đảm bảo cách đoạn khi xuất file PDF.

3. Gắn nội dung đã viết lại vào từng trường tương ứng trong một cấu trúc JSON mới.

    - Mỗi trường là một chuỗi (`str`) có định dạng xuống dòng rõ ràng.
    - Ưu tiên dùng `\\n\\n` để tách các đoạn văn khi cần truyền nội dung qua `create_to_trinh`.

4. In ra JSON kết quả để kiểm tra lỗi định dạng nếu cần.

5. Cuối cùng, gọi `create_to_trinh(...)` để tạo file `totrinh.pdf`.

Khi có yêu cầu chỉnh sửa lại, nhiệm vụ của bạn là vào trường cần xử lý theo yêu cầu người dùng và chỉnh sửa lại nội dung theo ý muốn của họ. Lúc này chỉ cần thực hiện chỉnh lại nội dung trường yêu cầu và làm tiếp step 4 và step 5

</guideline>

✅ Yêu cầu đặc biệt:

- Văn phong nghiêm túc, đúng chuẩn ngôn ngữ báo cáo tài chính – tín dụng doanh nghiệp.
- Tuyệt đối không để các đoạn văn bị dính liền nhau; phải cách đoạn rõ ràng bằng `\\n\\n`.
- Tất cả nội dung đầu ra phải bằng **tiếng Việt**.


"""
)




c:\Users\Song\anaconda3\envs\aws\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Song\anaconda3\envs\aws\lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value 5486400 is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


## TỜ TRÌNH

In [3]:
simple_agent("Hãy viết tờ trình thể hiện đầy đủ nội dung giúp tôi")

Tôi sẽ giúp bạn tạo một tờ trình đầy đủ bằng cách thu thập thông tin và tổng hợp theo từng phần.

1. Đầu tiên, tôi sẽ truy xuất thông tin từ cơ sở dữ liệu:
Tool #1: retrieve

Tool #2: retrieve

Tool #3: retrieve

Tool #4: retrieve
Dựa trên thông tin thu thập được, tôi sẽ tổng hợp một tờ trình đầy đủ:
Tool #5: create_to_trinh
content/fin_analyst/fin_charts/Bảng tài chính tổng hơp.png
✅ Ảnh đã được tải từ S3 và chèn vào Word.
content/fin_analyst/fin_charts/Cơ cấu vốn.png
✅ Ảnh đã được tải từ S3 và chèn vào ô bảng.
content/fin_analyst/fin_charts/Cơ cấu tài sản nợ.png
✅ Ảnh đã được tải từ S3 và chèn vào ô bảng.
content/fin_analyst/fin_charts/Biên lợi nhuận và lợi nhuận.png
✅ Ảnh đã được tải từ S3 và chèn vào ô bảng.
content/fin_analyst/fin_charts/Phân tích lợi nhuận.png
✅ Ảnh đã được tải từ S3 và chèn vào ô bảng.
content/fin_analyst/fin_charts/Tài sản và hiệu quả sinh lời.png
✅ Ảnh đã được tải từ S3 và chèn vào ô bảng.
content/fin_analyst/fin_charts/Vốn chủ sở hữu và khả năng sinh lời.png


100%|██████████| 1/1 [00:04<00:00,  4.26s/it]


Tôi đã tạo một tờ trình đầy đủ dựa trên thông tin thu thập được. Tờ trình bao gồm các phần chính:

1. Thông tin khách hàng
2. Lịch sử phát triển
3. Cơ cấu tổ chức và ban lãnh đạo
4. Đánh giá thị trường
5. Đánh giá sản phẩm/dịch vụ
6. Tình trạng kiểm toán
7. Phân tích tài chính chi tiết
8. Phân tích rủi ro và giải pháp
9. Kết luận và đề xuất

Tờ trình đã được tạo thành công dưới dạng file PDF. Bạn có thể xem xét nội dung và yêu cầu chỉnh sửa bất kỳ phần nào nếu cần.

Bạn có muốn tôi điều chỉnh hoặc bổ sung thêm thông tin nào không?

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'Tôi đã tạo một tờ trình đầy đủ dựa trên thông tin thu thập được. Tờ trình bao gồm các phần chính:\n\n1. Thông tin khách hàng\n2. Lịch sử phát triển\n3. Cơ cấu tổ chức và ban lãnh đạo\n4. Đánh giá thị trường\n5. Đánh giá sản phẩm/dịch vụ\n6. Tình trạng kiểm toán\n7. Phân tích tài chính chi tiết\n8. Phân tích rủi ro và giải pháp\n9. Kết luận và đề xuất\n\nTờ trình đã được tạo thành công dưới dạng file PDF. Bạn có thể xem xét nội dung và yêu cầu chỉnh sửa bất kỳ phần nào nếu cần.\n\nBạn có muốn tôi điều chỉnh hoặc bổ sung thêm thông tin nào không?'}]}, metrics=EventLoopMetrics(cycle_count=6, tool_metrics={'retrieve': ToolMetrics(tool={'toolUseId': 'tooluse_LNC2QiP8QHmmtslsYru2ZQ', 'name': 'retrieve', 'input': {'text': 'Tình trạng kiểm toán báo cáo tài chính, đơn vị kiểm toán và nội dung nổi bật của báo cáo'}}, call_count=4, success_count=4, error_count=0, total_time=1.85235595703125), 'create_to_trinh'

## CHỈNH SỬA

In [ ]:
simple_agent("Hãy cập nhật thêm thông tin ông Mai Lê Lợi có 10 năm kinh nghiệm trong ngành Logistics đạt được nhiều giải thưởng trong và ngoài nước")

NameError: name 'agent' is not defined